In [1]:
from dplython import select, DplyFrame, X, arrange, count, sift, head, summarize, group_by, tail, mutate
import numpy as np
import pandas as pd
import string
import rcurl

In [2]:
date_of_study = "04-04-2020"
# Historical data
covid_hist = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-30-2020.csv")

covid_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3439 entries, 0 to 3438
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FIPS            3148 non-null   float64
 1   Admin2          3178 non-null   object 
 2   Province_State  3265 non-null   object 
 3   Country_Region  3439 non-null   object 
 4   Last_Update     3439 non-null   object 
 5   Lat             3439 non-null   float64
 6   Long_           3439 non-null   float64
 7   Confirmed       3439 non-null   int64  
 8   Deaths          3439 non-null   int64  
 9   Recovered       3439 non-null   int64  
 10  Active          3439 non-null   int64  
 11  Combined_Key    3439 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 322.5+ KB


In [3]:
covid_us_hist = covid_hist[(covid_hist['Country_Region'] == "US") & (covid_hist['FIPS'].isnull() == False)]
covid_us_hist.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,3/30/20 22:52,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,3/30/20 22:52,30.295065,-92.414197,11,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,3/30/20 22:52,37.767072,-75.632346,6,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,3/30/20 22:52,43.452658,-116.241552,113,2,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,3/30/20 22:52,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [4]:
# Import outcome data from JHU CSSE
covid = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(date_of_study))

#covid = read.csv(text=getURL(paste0("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/",date_of_study,".csv")))

covid_us = covid[(covid['Country_Region'] == "US") & (covid['FIPS'].isnull() == True)]
covid_us1 = covid_us_hist[(covid_us_hist['FIPS'].isin(covid_us)) & (covid_us_hist['Confirmed'] == 0) & (covid_us_hist['Deaths'] == 0) & (covid_us_hist['FIPS'].isnull() == False)]

covid_us = covid_us.append(covid_us1)

In [5]:
# Import exposure PM2.5 data
county_pm = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/county_pm25.csv")
county_temp = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/temp_seasonal_county.csv")

In [9]:
# Import census, brfss, testing, mortality, hosptial beds data as potential confounders
county_census = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/census_county_interpolated.csv")
county_brfss = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/brfss_county_interpolated.csv")
state_test = pd.read_csv("https://covidtracking.com/api/states/daily.csv")
state_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      2340 non-null   int64  
 1   state                     2340 non-null   object 
 2   positive                  2325 non-null   float64
 3   negative                  2184 non-null   float64
 4   pending                   552 non-null    float64
 5   hospitalizedCurrently     495 non-null    float64
 6   hospitalizedCumulative    713 non-null    float64
 7   inIcuCurrently            230 non-null    float64
 8   inIcuCumulative           118 non-null    float64
 9   onVentilatorCurrently     194 non-null    float64
 10  onVentilatorCumulative    36 non-null     float64
 11  recovered                 460 non-null    float64
 12  hash                      2340 non-null   object 
 13  dateChecked               2340 non-null   object 
 14  death   

In [14]:
date_clean = date_of_study.replace("-","",-1)
year = date_clean[4:8]
month_day = date_clean[0:4]
state_test = state_test[state_test['date'] == "{}{}".format('2020','0404')].drop(columns='fips')
#state_test = subset(state_test, date ==paste0(substring(str_remove_all(date_of_study, "-"),5,8),substring(str_remove_all(date_of_study, "-"),1,4)))[,-20]
statecode = pd.read_csv("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/statecode.csv")

/home/angelo/anaconda3/envs/flatiron/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [15]:
state_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      0 non-null      int64  
 1   state                     0 non-null      object 
 2   positive                  0 non-null      float64
 3   negative                  0 non-null      float64
 4   pending                   0 non-null      float64
 5   hospitalizedCurrently     0 non-null      float64
 6   hospitalizedCumulative    0 non-null      float64
 7   inIcuCurrently            0 non-null      float64
 8   inIcuCumulative           0 non-null      float64
 9   onVentilatorCurrently     0 non-null      float64
 10  onVentilatorCumulative    0 non-null      float64
 11  recovered                 0 non-null      float64
 12  hash                      0 non-null      object 
 13  dateChecked               0 non-null      object 
 14  death                 

In [26]:
hospitals = pd.read_csv("https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.csv?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D")
hospitals['BEDS'][hospitals['BEDS'] < 0] = 0

/home/angelo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
county_base_mortality = pd.read_table("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/county_base_mortality.txt", sep='\t')
county_old_mortality = pd.read_table("https://raw.githubusercontent.com/wxwx1993/PM_COVID/master/Data/county_old_mortality.txt",  sep='\t')
county_old_mortality.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 3145 entries, nan to nan
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   County       3145 non-null   object 
 1   County Code  3145 non-null   int64  
 2   Deaths       3145 non-null   int64  
 3   Population   3145 non-null   int64  
 4   Crude Rate   3145 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 147.4+ KB


In [41]:
county_old_mortality=county_old_mortality.rename(columns={"Population": "older_Population"})
county_old_mortality.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 3145 entries, nan to nan
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   County            3145 non-null   object 
 1   County Code       3145 non-null   int64  
 2   Deaths            3145 non-null   int64  
 3   older_Population  3145 non-null   int64  
 4   Crude Rate        3145 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 147.4+ KB


In [40]:
county_base_mortality = pd.merge(county_base_mortality, county_old_mortality.iloc[:, [2,4]], how='left', on = "County Code")
county_base_mortality['older_pecent'] = county_base_mortality['older_Population']/county_base_mortality['Population']

KeyError: 'County Code'

In [ ]:
# merging data
state_test = merge(state_test,statecode,by.x = "state" ,by.y = "Code" )

In [ ]:
# pm average over 17 years
county_temp_aggregated = county_temp %>% 
  group_by(fips) %>% 
  summarise(mean_winter_temp= mean(winter_tmmx),
            mean_summer_temp= mean(summer_tmmx),
            mean_winter_rm= mean(winter_rmax),
            mean_summer_rm= mean(summer_rmax))

In [ ]:
county_pm_aggregated = merge(county_pm_aggregated,county_temp_aggregated,by="fips",all.x = T)


In [ ]:
county_hospitals_aggregated = hospitals %>%
  group_by(COUNTYFIPS) %>%
  summarise(beds = sum(BEDS))


In [ ]:
county_census_aggregated2 = county_census[county_census['year']==2016]
county_census_aggregated2['q_popdensity'] = 1
quantile_popdensity = quantile(county_census_aggregated2['popdensity,c(0.25,0.5,0.75))
county_census_aggregated2$q_popdensity[county_census_aggregated2$popdensity<=quantile_popdensity[1]] = 1
county_census_aggregated2$q_popdensity[county_census_aggregated2$popdensity>quantile_popdensity[1] &
                                         county_census_aggregated2$popdensity<=quantile_popdensity[2]] = 2
county_census_aggregated2$q_popdensity[county_census_aggregated2$popdensity>quantile_popdensity[2] &
                                         county_census_aggregated2$popdensity<=quantile_popdensity[3]] = 3
county_census_aggregated2$q_popdensity[county_census_aggregated2$popdensity>quantile_popdensity[3]] = 4

county_brfss_aggregated = subset(county_brfss, year==2012)

county_census_aggregated2 = merge(county_census_aggregated2,county_brfss_aggregated,
                                  by="fips",all.x=T)

aggregate_pm = merge(county_pm_aggregated,covid_us,by.x="fips",by.y = "FIPS")

aggregate_pm_census = merge(aggregate_pm,county_census_aggregated2,by.x="fips",by.y = "fips")

aggregate_pm_census_cdc = merge(aggregate_pm_census,county_base_mortality[,c(1,4:5,8:9)],by.x = "fips",by.y = "County.Code",all.x = T)

aggregate_pm_census_cdc = aggregate_pm_census_cdc[is.na(aggregate_pm_census_cdc$fips) ==F,]

aggregate_pm_census_cdc_test = merge(aggregate_pm_census_cdc,state_test,by.x="Province_State",by.y = "State")
aggregate_pm_census_cdc_test = aggregate_pm_census_cdc_test %>%
  group_by(Province_State) %>%
  mutate(population_frac_county = population/sum(population),
         totalTestResults_county = population_frac_county*totalTestResults)


aggregate_pm_census_cdc_test_beds = merge(aggregate_pm_census_cdc_test,county_hospitals_aggregated,by.x = "fips",by.y = "COUNTYFIPS",all.x = T)